# Final Round of Transformation
String matching with a list of city names from a dataset retrieved from here:
https://www.kaggle.com/datasets/crisparada/brazilian-cities?resource=download


In [3]:
import os
import pandas as pd
import numpy as np
import time
import sqlalchemy as db
from dotenv import load_dotenv
from thefuzz import process, fuzz
from datetime import date

In [4]:
# Load environment variables from .env file
load_dotenv()

# Access environment variable
db_local=os.environ['olist_db_local']
db_azure=os.environ['olist_db_azure']
back4app_id=os.environ['back4app_id']
back4app_key=os.environ['back4app_key']
export_path_local=os.environ['export_path_local']

# Import and Inspect Processed Location Datasets

In [4]:
# Download processed location data, convert to a dataframe
geo_df = pd.read_csv(f'{export_path_local}geolocation_2024-04-28.csv')
geo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 720154 entries, 0 to 720153
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Unnamed: 0       720154 non-null  int64  
 1   zip_code_prefix  720154 non-null  int64  
 2   lat              720154 non-null  float64
 3   lng              720154 non-null  float64
 4   city             720154 non-null  object 
 5   state            720154 non-null  object 
 6   cep              689834 non-null  float64
 7   district         637088 non-null  object 
dtypes: float64(3), int64(2), object(3)
memory usage: 44.0+ MB


In [5]:
# Look at first 5 entries
geo_df.head()

,Unnamed: 0,zip_code_prefix,lat,lng,city,state,cep,district
0,0,1001,-23.549292,-46.633559,São Paulo,SP,1001902.0,Sé
1,1,1001,-23.550498,-46.634338,São Paulo,SP,1001902.0,Sé
2,2,1001,-23.550642,-46.634410,São Paulo,SP,1001902.0,Sé
3,3,1001,-23.549698,-46.633909,São Paulo,SP,1001902.0,Sé
4,4,1001,-23.550263,-46.634196,São Paulo,SP,1001902.0,Sé


In [7]:
# Zip code prefix should be 5 digits long with leading zeroes
geo_df['zip_code_prefix'] = geo_df['zip_code_prefix'].apply(lambda x: '{0:0>5}'.format(x))

In [8]:
# Drop old index col
geo_df.drop('Unnamed: 0', axis=1, inplace=True)

In [37]:
# Drop cep col - not accurate after merge due to incomplete original zip code data
geo_df.drop('cep', axis=1, inplace=True)
geo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 720154 entries, 0 to 720153
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   zip_code_prefix  720154 non-null  object 
 1   lat              720154 non-null  float64
 2   lng              720154 non-null  float64
 3   city             720154 non-null  object 
 4   state            720154 non-null  object 
 5   district         637088 non-null  object 
dtypes: float64(2), object(4)
memory usage: 33.0+ MB


In [9]:
# Inspect unique values in city (1)
city = geo_df['city'].unique().tolist()
print('Unique city values: ', geo_df['city'].nunique(), '\n', city)

Unique city values:  5497 
 ['São Paulo', 'sao paulo', 'Osasco', 'osasco', 'Carapicuíba', 'carapicuiba', 'Barueri', 'barueri', 'Santana de Parnaíba', 'Pirapora do Bom Jesus', 'Jandira', 'jandira', 'itapevi', 'Itapevi', 'Cotia', 'Vargem Grande Paulista', 'vargem grande paulista', 'Taboão da Serra', 'Embu das Artes', 'Itapecerica da Serra', 'São Lourenço da Serra', 'Embu-Guaçu', 'Juquitiba', 'Guarulhos', 'guarulhos', 'Arujá', 'Santa Isabel', 'Mairiporã', 'Caieiras', 'caieiras', 'Cajamar', 'jordanesia', 'polvilho', 'cajamar', 'Franco da Rocha', 'Francisco Morato', 'Ferraz de Vasconcelos', 'Poá', 'Itaquaquecetuba', 'Suzano', 'suzano', 'Mogi das Cruzes', 'Guararema', 'Biritiba-Mirim', 'Salesópolis', 'Santo André', 'santo andre', 'Mauá', 'maua', 'Ribeirão Pires', 'Rio Grande da Serra', 'São Caetano do Sul', 'São Bernardo do Campo', 'sao bernardo do campo', 'Diadema', 'Santos', 'santos', 'Bertioga', 'São Vicente', 'sao vicente', 'Guarujá', 'Cubatão', 'São Sebastião', 'maresias', 'Ilhabela', '

# Import & Inspect City Dataset

In [15]:
# Import list of Brazillian cities
cities_df = pd.read_csv(f'{export_path_local}BRAZIL_CITIES_REV2022.CSV')
cities_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5578 entries, 0 to 5577
Data columns (total 81 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   CITY                    5578 non-null   object 
 1   STATE                   5578 non-null   object 
 2   CAPITAL                 5578 non-null   int64  
 3   IBGE_RES_POP            5578 non-null   int64  
 4   IBGE_RES_POP_BRAS       5578 non-null   int64  
 5   IBGE_RES_POP_ESTR       5578 non-null   int64  
 6   IBGE_DU                 5578 non-null   int64  
 7   IBGE_DU_URBAN           5578 non-null   int64  
 8   IBGE_DU_RURAL           5578 non-null   int64  
 9   IBGE_POP                5578 non-null   int64  
 10  IBGE_1                  5578 non-null   int64  
 11  IBGE_1-4                5578 non-null   int64  
 12  IBGE_5-9                5578 non-null   int64  
 13  IBGE_10-14              5578 non-null   int64  
 14  IBGE_15-59              5578 non-null   

In [16]:
# Create a list of city names
cities = cities_df['CITY'].tolist()
print(len(cities), '\n', cities)

5578 
 ['Abadia De Goiás', 'Abadia Dos Dourados', 'Abadiânia', 'Abaetetuba', 'Abaeté', 'Abaiara', 'Abaré', 'Abatiá', 'Abaíra', 'Abdon Batista', 'Abel Figueiredo', 'Abelardo Luz', 'Abre Campo', 'Abreu E Lima', 'Abreulândia', 'Acaiaca', 'Acajutiba', 'Acarape', 'Acaraú', 'Acari', 'Acará', 'Acauã', 'Aceguá', 'Acopiara', 'Acorizal', 'Acrelândia', 'Acreúna', 'Adamantina', 'Adelândia', 'Adolfo', 'Adrianópolis', 'Adustina', 'Afogados Da Ingazeira', 'Afonso Bezerra', 'Afonso Cláudio', 'Afonso Cunha', 'Afrânio', 'Afuá', 'Agrestina', 'Agricolândia', 'Agrolândia', 'Agronômica', 'Aguanil', 'Aguaí', 'Agudo', 'Agudos', 'Agudos Do Sul', 'Aguiar', 'Aguiarnópolis', 'Aimorés', 'Aiquara', 'Aiuaba', 'Aiuruoca', 'Ajuricaba', 'Alagoa', 'Alagoa Grande', 'Alagoa Nova', 'Alagoinha', 'Alagoinha', 'Alagoinha Do Piauí', 'Alagoinhas', 'Alambari', 'Albertina', 'Alcantil', 'Alcinópolis', 'Alcobaça', 'Alcântara', 'Alcântaras', 'Aldeias Altas', 'Alecrim', 'Alegre', 'Alegrete', 'Alegrete Do Piauí', 'Alegria', 'Alenquer'

In [25]:
# Test for names in list
if 'São Paulo' in cities:
    print("Desired item is in list")

Desired item is in list


In [26]:
# Test matching ratio
fuzz.WRatio('São Paulo', 'Sao Paulo')

94

In [22]:
# Inspect match statement output
matches = process.extract('Abadia Dos Dourados', geo_df['city'], limit = geo_df.shape[0])
matches

[('Abadia Dos Dourados', 100, 450536),
 ('Abadia Dos Dourados', 100, 450537),
 ('Abadia Dos Dourados', 100, 450538),
 ('Abadia Dos Dourados', 100, 450539),
 ('Abadia Dos Dourados', 100, 450540),
 ('Abadia Dos Dourados', 100, 450541),
 ('Abadia Dos Dourados', 100, 450542),
 ('Abadia Dos Dourados', 100, 450543),
 ('Abadia Dos Dourados', 100, 450544),
 ('Abadia Dos Dourados', 100, 450545),
 ('Abadia Dos Dourados', 100, 450546),
 ('Abadia Dos Dourados', 100, 450547),
 ('Abadia Dos Dourados', 100, 450548),
 ('Abadia Dos Dourados', 100, 450549),
 ('Abadia Dos Dourados', 100, 450550),
 ('Abadia Dos Dourados', 100, 450551),
 ('Abadia Dos Dourados', 100, 450552),
 ('Abadia Dos Dourados', 100, 450553),
 ('Abadia Dos Dourados', 100, 450554),
 ('Abadia Dos Dourados', 100, 450555),
 ('Abadia Dos Dourados', 100, 450556),
 ('Dourado', 90, 202141),
 ('Dourado', 90, 202142),
 ('Dourado', 90, 202143),
 ('Dourado', 90, 202144),
 ('Dourado', 90, 202145),
 ('Dourado', 90, 202146),
 ('Dourado', 90, 202147),

# Using City dataset, perform fuzzy string-match with processed location dataset

In [23]:
%%time
for i, city in enumerate(cities):
    matches = process.extract(city, geo_df['city'], limit=geo_df.shape[0])
    high_score_match = None

    # Iterate through the matches to find the one with the highest score above 90
    for potential_match in matches:
        if potential_match[1] > 91:
            if high_score_match is None or potential_match[1] > high_score_match[1]:
                high_score_match = potential_match

    # If a match with the highest score above 80 is found, update the 'geolocation_city'
    if high_score_match:
        print(high_score_match[0])
        geo_df.loc[geo_df['city'] == high_score_match[0], 'city'] = city
    n = geo_df[geo_df['city'] == city]['city'].count()
    print(f'{i}', '. ', city, '\t','Matched: ', n)

Abadia De Goiás
0 .  Abadia De Goiás 	 Matched:  7
Abadia Dos Dourados
1 .  Abadia Dos Dourados 	 Matched:  21
Abadiânia
2 .  Abadiânia 	 Matched:  23
Abaetetuba
3 .  Abaetetuba 	 Matched:  125
Abaeté
4 .  Abaeté 	 Matched:  109
Abaiara
5 .  Abaiara 	 Matched:  4
Abaré
6 .  Abaré 	 Matched:  22
Abatiá
7 .  Abatiá 	 Matched:  22
Abaíra
8 .  Abaíra 	 Matched:  10
Abdon Batista
9 .  Abdon Batista 	 Matched:  10
Abel Figueiredo
10 .  Abel Figueiredo 	 Matched:  15
Abelardo Luz
11 .  Abelardo Luz 	 Matched:  47
Abre Campo
12 .  Abre Campo 	 Matched:  34
Abreu E Lima
13 .  Abreu E Lima 	 Matched:  112
Abreulândia
14 .  Abreulândia 	 Matched:  1
Acaiaca
15 .  Acaiaca 	 Matched:  15
Acajutiba
16 .  Acajutiba 	 Matched:  18
Acarape
17 .  Acarape 	 Matched:  6
Acaraú
18 .  Acaraú 	 Matched:  38
Acari
19 .  Acari 	 Matched:  15
Acará
20 .  Acará 	 Matched:  8
Acauã
21 .  Acauã 	 Matched:  4
Aceguá
22 .  Aceguá 	 Matched:  7
Acopiara
23 .  Acopiara 	 Matched:  44
Acorizal
24 .  Acorizal 	 Matched:

Ananás
183 .  Ananás 	 Matched:  19
Anapu
184 .  Anapu 	 Matched:  27
Anapurus
185 .  Anapurus 	 Matched:  8
Anastácio
186 .  Anastácio 	 Matched:  39
Anaurilândia
187 .  Anaurilândia 	 Matched:  22
Anchieta
188 .  Anchieta 	 Matched:  83
Anchieta
189 .  Anchieta 	 Matched:  83
Andaraí
190 .  Andaraí 	 Matched:  14
Andirá
191 .  Andirá 	 Matched:  87
Andorinha
192 .  Andorinha 	 Matched:  22
Andradas
193 .  Andradas 	 Matched:  156
Andradina
194 .  Andradina 	 Matched:  467
Andrelândia
195 .  Andrelândia 	 Matched:  64
André Da Rocha
196 .  André Da Rocha 	 Matched:  3
Angatuba
197 .  Angatuba 	 Matched:  89
Angelim
198 .  Angelim 	 Matched:  12
Angelina
199 .  Angelina 	 Matched:  8
Angelândia
200 .  Angelândia 	 Matched:  13
Angical
201 .  Angical 	 Matched:  7
Angical Do Piauí
202 .  Angical Do Piauí 	 Matched:  5
Angico
203 .  Angico 	 Matched:  3
Angicos
204 .  Angicos 	 Matched:  19
Angra Dos Reis
205 .  Angra Dos Reis 	 Matched:  769
Anguera
206 .  Anguera 	 Matched:  6
Angélica

Assunção Do Piauí
378 .  Assunção Do Piauí 	 Matched:  3
Astolfo Dutra
379 .  Astolfo Dutra 	 Matched:  53
Astorga
380 .  Astorga 	 Matched:  89
Atalaia
381 .  Atalaia 	 Matched:  25
Atalaia
382 .  Atalaia 	 Matched:  25
Atalaia Do Norte
383 .  Atalaia Do Norte 	 Matched:  4
Atalanta
384 .  Atalanta 	 Matched:  11
Ataléia
385 .  Ataléia 	 Matched:  27
Atibaia
386 .  Atibaia 	 Matched:  988
Atílio Vivácqua
387 .  Atílio Vivacqua 	 Matched:  18
Augustinópolis
388 .  Augustinópolis 	 Matched:  39
Augusto Corrêa
389 .  Augusto Corrêa 	 Matched:  21
Augusto De Lima
390 .  Augusto De Lima 	 Matched:  12
Augusto Pestana
391 .  Augusto Pestana 	 Matched:  28
392 .  Augusto Severo 	 Matched:  0
Aurelino Leal
393 .  Aurelino Leal 	 Matched:  7
Auriflama
394 .  Auriflama 	 Matched:  44
Aurilândia
395 .  Aurilândia 	 Matched:  7
Aurora
396 .  Aurora 	 Matched:  30
Aurora
397 .  Aurora 	 Matched:  30
Aurora Do Pará
398 .  Aurora Do Pará 	 Matched:  8
Aurora Do Tocantins
399 .  Aurora Do Tocantins 	

Belo Oriente
553 .  Belo Oriente 	 Matched:  79
Belo Vale
554 .  Belo Vale 	 Matched:  23
Belterra
555 .  Belterra 	 Matched:  7
Belágua
556 .  Belágua 	 Matched:  2
Belém
557 .  Belém 	 Matched:  2873
Belém
558 .  Belém 	 Matched:  2873
Belém
559 .  Belém 	 Matched:  2873
Belém De Maria
560 .  Belém De Maria 	 Matched:  6
Belém Do Brejo Do Cruz
561 .  Belém Do Brejo Do Cruz 	 Matched:  3
Belém Do Piauí
562 .  Belém Do Piauí 	 Matched:  2
Belém Do São Francisco
563 .  Belém Do São Francisco 	 Matched:  20
Beneditinos
564 .  Beneditinos 	 Matched:  5
Benedito Leite
565 .  Benedito Leite 	 Matched:  3
Benedito Novo
566 .  Benedito Novo 	 Matched:  26
Benevides
567 .  Benevides 	 Matched:  29
Benjamin Constant
568 .  Benjamin Constant 	 Matched:  7
Benjamin Constant Do Sul
569 .  Benjamin Constant Do Sul 	 Matched:  5
Bento De Abreu
570 .  Bento De Abreu 	 Matched:  14
Bento Fernandes
571 .  Bento Fernandes 	 Matched:  1
Bento Gonçalves
572 .  Bento Gonçalves 	 Matched:  559
Bequimão
573 

Braço Do Trombudo
724 .  Braço Do Trombudo 	 Matched:  15
Braúna
725 .  Braúna 	 Matched:  11
Braúnas
726 .  Braúnas 	 Matched:  10
Brejetuba
727 .  Brejetuba 	 Matched:  19
Brejinho
728 .  Brejinho 	 Matched:  8
Brejinho
729 .  Brejinho 	 Matched:  8
Brejinho De Nazaré
730 .  Brejinho De Nazaré 	 Matched:  11
Brejão
731 .  Brejo 	 Matched:  24
Brejo Alegre
732 .  Brejo Alegre 	 Matched:  8
Brejo Da Madre De Deus
733 .  Brejo Da Madre De Deus 	 Matched:  31
Brejo De Areia
734 .  Brejo De Areia 	 Matched:  1
Brejo Do Cruz
735 .  Brejo Do Cruz 	 Matched:  16
Brejo Do Piauí
736 .  Brejo Do Piauí 	 Matched:  2
Brejo Dos Santos
737 .  Brejo Dos Santos 	 Matched:  9
Brejo Grande
738 .  Brejo Grande 	 Matched:  2
Brejo Grande Do Araguaia
739 .  Brejo Grande Do Araguaia 	 Matched:  7
Brejo Santo
740 .  Brejo Santo 	 Matched:  60
Brejolândia
741 .  Brejolândia 	 Matched:  5
Brejo
742 .  Brejão 	 Matched:  24
Brejões
743 .  Brejões 	 Matched:  9
Breu Branco
744 .  Breu Branco 	 Matched:  30
Brev

...

São José Do Barreiro
4855 .  São José Do Barreiro 	 Matched:  20
São José Do Belmonte
4856 .  São José Do Belmonte 	 Matched:  31
São José Do Bonfim
4857 .  São José Do Bonfim 	 Matched:  3
São José Do Brejo Do Cruz
4858 .  São José Do Brejo Do Cruz 	 Matched:  2
São José Do Calçado
4859 .  São José Do Calçado 	 Matched:  36
São José Do Campestre
4860 .  São José Do Campestre 	 Matched:  5
São José Do Cedro
4861 .  São José Do Cedro 	 Matched:  70
São José Do Cerrito
4862 .  São José Do Cerrito 	 Matched:  16
São José Do Divino
4863 .  São José Do Divino 	 Matched:  15
São José Do Divino
4864 .  São José Do Divino 	 Matched:  15
São José Do Egito
4865 .  São José Do Egito 	 Matched:  55
São José Do Goiabal
4866 .  São José Do Goiabal 	 Matched:  21
São José Do Herval
4867 .  São José Do Herval 	 Matched:  9
São José Do Hortêncio
4868 .  São José Do Hortêncio 	 Matched:  17
São José Do Inhacorá
4869 .  São José Do Inhacorá 	 Matched:  5
São José Do Jacuri
4870 .  São José Do Jacuri 	 Ma

Sao Miguel Do Passa Quatro
4990 .  São Miguel Do Passa Quatro 	 Matched:  5
São Miguel Do Tapuio
4991 .  São Miguel Do Tapuio 	 Matched:  6
São Miguel Do Tocantins
4992 .  São Miguel Do Tocantins 	 Matched:  7
São Miguel Dos Campos
4993 .  São Miguel Dos Campos 	 Matched:  46
São Miguel Dos Milagres
4994 .  São Miguel Dos Milagres 	 Matched:  5
São Nicolau
4995 .  São Nicolau 	 Matched:  10
São Patrício
4996 .  São Patrício 	 Matched:  6
São Paulo
4997 .  São Paulo 	 Matched:  91768
São Paulo Das Missões
4998 .  São Paulo Das Missões 	 Matched:  21
São Paulo De Olivença
4999 .  São Paulo De Olivença 	 Matched:  2
São Paulo Do Potengi
5000 .  São Paulo Do Potengi 	 Matched:  15
São Pedro
5001 .  São Pedro 	 Matched:  173
São Pedro
5002 .  São Pedro 	 Matched:  173
Sao Pedro Da Aldeia
5003 .  São Pedro Da Aldeia 	 Matched:  411
São Pedro Da Cipa
5004 .  São Pedro Da Cipa 	 Matched:  7
São Pedro Da Serra
5005 .  São Pedro Da Serra 	 Matched:  10
São Pedro Da União
5006 .  São Pedro Da Uni

Taquaruçu Do Sul
5146 .  Taquaruçu Do Sul 	 Matched:  8
Tarabai
5147 .  Tarabai 	 Matched:  31
Tarauacá
5148 .  Tarauacá 	 Matched:  26
Tarrafas
5149 .  Tarrafas 	 Matched:  1
5150 .  Tartarugalzinho 	 Matched:  0
Tarumirim
5151 .  Tarumirim 	 Matched:  52
Tarumã
5152 .  Tarumã 	 Matched:  74
Tasso Fragoso
5153 .  Tasso Fragoso 	 Matched:  7
Tatuí
5154 .  Tatuí 	 Matched:  305
Taubaté
5155 .  Taubaté 	 Matched:  1997
Tauá
5156 .  Tauá 	 Matched:  37
Tavares
5157 .  Tavares 	 Matched:  39
Tavares
5158 .  Tavares 	 Matched:  39
Tefé
5159 .  Tefé 	 Matched:  20
Teixeira
5160 .  Teixeira 	 Matched:  13
Teixeira De Freitas
5161 .  Teixeira De Freitas 	 Matched:  345
Teixeira Soares
5162 .  Teixeira Soares 	 Matched:  16
Teixeiras
5163 .  Teixeiras 	 Matched:  32
Teixeirópolis
5164 .  Teixeirópolis 	 Matched:  5
Tejupá
5165 .  Tejupá 	 Matched:  11
Tejuçuoca
5166 .  Tejuçuoca 	 Matched:  6
Telha
5167 .  Telha 	 Matched:  1
Telêmaco Borba
5168 .  Telêmaco Borba 	 Matched:  408
Tenente Ananias

Uchoa
5332 .  Uchoa 	 Matched:  49
Uibaí
5333 .  Uibaí 	 Matched:  21
5334 .  Uiramutã 	 Matched:  0
Uirapuru
5335 .  Uirapuru 	 Matched:  2
Uiraúna
5336 .  Uiraúna 	 Matched:  17
Ulianópolis
5337 .  Ulianópolis 	 Matched:  23
Umari
5338 .  Umari 	 Matched:  8
Umarizal
5339 .  Umarizal 	 Matched:  13
Umbaúba
5340 .  Umbaúba 	 Matched:  29
Umburanas
5341 .  Umburanas 	 Matched:  9
Umburatiba
5342 .  Umburatiba 	 Matched:  13
Umbuzeiro
5343 .  Umbuzeiro 	 Matched:  7
Umirim
5344 .  Umirim 	 Matched:  2
Umuarama
5345 .  Umuarama 	 Matched:  424
Unaí
5346 .  Una 	 Matched:  563
Una
5347 .  Unaí 	 Matched:  563
Uniflor
5348 .  Uniflor 	 Matched:  9
Unistalda
5349 .  Unistalda 	 Matched:  4
União
5350 .  União 	 Matched:  12
União Da Serra
5351 .  União Da Serra 	 Matched:  5
União Da Vitória
5352 .  União Da Vitória 	 Matched:  281
União De Minas
5353 .  União De Minas 	 Matched:  12
União Do Oeste
5354 .  União Do Oeste 	 Matched:  5
União Do Sul
5355 .  União Do Sul 	 Matched:  16
União D

Várzea Da Palma
5511 .  Várzea Da Palma 	 Matched:  112
Várzea Da Roça
5512 .  Várzea Da Roça 	 Matched:  15
Várzea Do Poço
5513 .  Várzea Do Poço 	 Matched:  14
Várzea Grande
5514 .  Várzea Grande 	 Matched:  309
Várzea Grande
5515 .  Várzea Grande 	 Matched:  309
Várzea Nova
5516 .  Várzea Nova 	 Matched:  16
Várzea Paulista
5517 .  Várzea Paulista 	 Matched:  263
Wagner
5518 .  Wagner 	 Matched:  9
Wall Ferraz
5519 .  Wall Ferraz 	 Matched:  4
Wanderley
5520 .  Wanderley 	 Matched:  14
Wanderlândia
5521 .  Wanderlândia 	 Matched:  8
Wenceslau Braz
5522 .  Wenceslau Braz 	 Matched:  85
Wenceslau Braz
5523 .  Wenceslau Braz 	 Matched:  85
Wenceslau Guimarães
5524 .  Wenceslau Guimarães 	 Matched:  12
Westfalia
5525 .  Westfália 	 Matched:  8
Witmarsum
5526 .  Witmarsum 	 Matched:  5
Xambioá
5527 .  Xambioá 	 Matched:  21
Xambrê
5528 .  Xambrê 	 Matched:  12
Xangri-Lá
5529 .  Xangri-Lá 	 Matched:  94
Xanxerê
5530 .  Xanxerê 	 Matched:  267
Xapuri
5531 .  Xapuri 	 Matched:  14
Xavantina

In [24]:
# Inspect unique 'city' values in processed df
city = geo_df['city'].unique().tolist()
print('Unique city values: ', geo_df['city'].nunique(), '\n', city)

Unique city values:  5314 
 ['São Paulo', 'Sao Paulo', 'Osasco', 'Carapicuíba', 'Carapicuiba', 'Barueri', 'Santana De Parnaíba', 'Pirapora Do Bom Jesus', 'Jandira', 'Itapevi', 'Cotia', 'Vargem Grande Paulista', 'Taboão Da Serra', 'Embu Das Artes', 'Itapecerica Da Serra', 'São Lourenço Da Serra', 'Embu-Guaçu', 'Juquitiba', 'Guarulhos', 'Arujá', 'Santa Isabel', 'Mairiporã', 'Caieiras', 'Cajamar', 'Jordanesia', 'Polvilho', 'Franco Da Rocha', 'Francisco Morato', 'Ferraz De Vasconcelos', 'Poá', 'Itaquaquecetuba', 'Suzano', 'Mogi Das Cruzes', 'Guararema', 'Biritiba Mirim', 'Salesópolis', 'Santo André', 'Santo Andre', 'Mauá', 'Maua', 'Ribeirão Pires', 'Rio Grande Da Serra', 'São Caetano Do Sul', 'São Bernardo Do Campo', 'Sao Bernardo Do Campo', 'Diadema', 'Santos', 'Bertioga', 'São Vicente', 'Sao Vicente', 'Guarujá', 'Cubatão', 'São Sebastião', 'Maresias', 'Ilhabela', 'Caraguatatuba', 'Ubatuba', 'Praia Grande', 'Mongaguá', 'Itanhaém', 'Peruíbe', 'Itariri', 'Pedro De Toledo', 'Juquiá', 'Miraca

In [27]:
# Replace 'Sao' with 'São' in the 'column_name'
geo_df['city'] = geo_df['city'].str.replace('Sao', 'São')

In [28]:
# Inspect list of cities
cities = geo_df['city'].unique().tolist()
cities.sort()
print('Unique city values: ', geo_df['city'].nunique(), '\n', cities)

Unique city values:  5308 
 ['São Paulo', 'Osasco', 'Carapicuíba', 'Carapicuiba', 'Barueri', 'Santana De Parnaíba', 'Pirapora Do Bom Jesus', 'Jandira', 'Itapevi', 'Cotia', 'Vargem Grande Paulista', 'Taboão Da Serra', 'Embu Das Artes', 'Itapecerica Da Serra', 'São Lourenço Da Serra', 'Embu-Guaçu', 'Juquitiba', 'Guarulhos', 'Arujá', 'Santa Isabel', 'Mairiporã', 'Caieiras', 'Cajamar', 'Jordanesia', 'Polvilho', 'Franco Da Rocha', 'Francisco Morato', 'Ferraz De Vasconcelos', 'Poá', 'Itaquaquecetuba', 'Suzano', 'Mogi Das Cruzes', 'Guararema', 'Biritiba Mirim', 'Salesópolis', 'Santo André', 'Santo Andre', 'Mauá', 'Maua', 'Ribeirão Pires', 'Rio Grande Da Serra', 'São Caetano Do Sul', 'São Bernardo Do Campo', 'Diadema', 'Santos', 'Bertioga', 'São Vicente', 'Guarujá', 'Cubatão', 'São Sebastião', 'Maresias', 'Ilhabela', 'Caraguatatuba', 'Ubatuba', 'Praia Grande', 'Mongaguá', 'Itanhaém', 'Peruíbe', 'Itariri', 'Pedro De Toledo', 'Juquiá', 'Miracatu', 'Registro', 'Sete Barras', 'Iguape', 'Ilha Compr

In [ ]:
# Perform a similarity check on adjacent elements
city_1 = []
city_2 = []

for i in range(len(cities)-1):
    match = fuzz.WRatio(cities[i], cities[i+1])
    if match > 92:
        city_1.append(cities[i])
        city_2.append(cities[i+1])

print(city_1, city_2)

In [ ]:
for i in range(len(city_1)):
    print(i,' ',city_1[i],'\n',city_2[i])

In [ ]:
# Remove different cities from the lists
index = [1, 4, 9, 38, 54, 60, 61, 64, 65, 68]
for i in sorted(index, reverse=True):
    city_1.pop(i)
    city_2.pop(i)

In [ ]:
# Replace similar city names with the same name 
geo_df['city'].replace(to_replace=city_1, value=city_2, inplace=True)

# FROM HERE

In [8]:
# # Download processed location data, convert to a dataframe
# geo_df = pd.read_csv(f'{export_path_local}geolocation_realFuzzy_2024-04-30.csv')
# # Zip code prefix should be 5 digits long with leading zeroes
# geo_df['zip_code_prefix'] = geo_df['zip_code_prefix'].apply(lambda x: '{0:0>5}'.format(x))
# # Drop old index col
# geo_df.drop('Unnamed: 0', axis=1, inplace=True)

In [10]:
geo_df.describe()

,zip_code_prefix,lat,lng
count,720154.000000,720154.000000,720154.000000
mean,38611.140120,-20.974809,-46.452045
std,30576.575761,5.918866,4.419726
min,1001.000000,-36.605374,-101.466766
25%,13043.000000,-23.601862,-48.912225
50%,29345.000000,-22.861875,-46.645727
75%,66053.000000,-19.916325,-43.786507
max,99990.000000,45.065933,121.105394


In [24]:
geo_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 720121 entries, 0 to 720153
Data columns (total 5 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   zip_code_prefix  720121 non-null  int64  
 1   lat              720121 non-null  float64
 2   lng              720121 non-null  float64
 3   city             720121 non-null  object 
 4   state            720121 non-null  object 
dtypes: float64(2), int64(1), object(2)
memory usage: 33.0+ MB


# Remove outliers in geolocation data:
- most Northern spot is at 5 deg 16′ 27.8″ N latitude
- most southern spot is at 33 deg, 45′ 04.21″ S Latitude.
- most Eastern spot is 34 deg, 47′ 35.33″ W Long
- most Western spot is at 73 deg, 58′ 58.19″W Long
- https://en.wikipedia.org/wiki/Geography_of_Brazil

In [20]:
# Removing some outliers
#Brazils most Northern spot is at 5 deg 16′ 27.8″ N latitude.;
geo_df = geo_df[geo_df['lat'] <= 5.27438888]
#it’s most Western spot is at 73 deg, 58′ 58.19″W Long.
geo_df = geo_df[geo_df['lng'] >= -73.98283055]
#It’s most southern spot is at 33 deg, 45′ 04.21″ S Latitude.
geo_df = geo_df[geo_df['lat'] >= -33.75116944]
#It’s most Eastern spot is 34 deg, 47′ 35.33″ W Long.
geo_df = geo_df[geo_df['lng'] <=  -34.79314722]

In [25]:
geo_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 720121 entries, 0 to 720153
Data columns (total 5 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   zip_code_prefix  720121 non-null  int64  
 1   lat              720121 non-null  float64
 2   lng              720121 non-null  float64
 3   city             720121 non-null  object 
 4   state            720121 non-null  object 
dtypes: float64(2), int64(1), object(2)
memory usage: 33.0+ MB


In [23]:
geo_df.describe()

,zip_code_prefix,lat,lng
count,720121.000000,720121.000000,720121.000000
mean,38610.417852,-20.976707,-46.453029
std,30576.731593,5.909591,4.408588
min,1001.000000,-33.692616,-72.930746
25%,13043.000000,-23.601866,-48.912230
50%,29345.000000,-22.861926,-46.645742
75%,66053.000000,-19.916486,-43.787134
max,99990.000000,4.482242,-34.793685


In [48]:
geo_df.nunique()

zip_code_prefix     19010
lat                717316
lng                717580
city                 5247
state                  27
dtype: int64

In [47]:
geo_df['state'] = geo_df['state'].str.strip()
geo_df['state'].nunique()

27

In [33]:
geo_df.head()

,zip_code_prefix,lat,lng,city,state
0,01001,-23.549292,-46.633559,São Paulo,SP
1,01001,-23.550498,-46.634338,São Paulo,SP
2,01001,-23.550642,-46.634410,São Paulo,SP
3,01001,-23.549698,-46.633909,São Paulo,SP
4,01001,-23.550263,-46.634196,São Paulo,SP


In [49]:
# Export to .csv file
today = date.today()
geo_df.to_csv(f'{export_path_local}geolocation_realFuzzy_{today}.csv')

# Create table in db

In [35]:
# Connect to DB
engine = db.create_engine(db_azure)
conn = engine.raw_connection()

In [36]:
# Create new table in PostgreSQL
commands = (f'''CREATE TABLE IF NOT EXISTS location(
);''')

# Initialize connection to PostgreSQL
cur = conn.cursor()

# Create cursor to execute SQL commands
#for command in commands:
cur.execute(commands)

# Commit changes
conn.commit()

In [37]:
# Copy data to table
geo_df.to_sql(name= 'location', con = engine, if_exists= 'append')

121

In [38]:
# Close communication with server
cur.close()
conn.close()

# End - Random String Checks

In [34]:
print(geo_df[geo_df['city'] == 'Abadia Dos Dourados']['city'].count())

21


In [35]:
for ele in sorted(city):
    print(ele)

Abadia De Goiás
Abadia Dos Dourados
Abadiânia
Abaetetuba
Abaeté
Abaiara
Abaré
Abatiá
Abaíra
Abdon Batista
Abel Figueiredo
Abelardo Luz
Abre Campo
Abreu E Lima
Abreulândia
Acaiaca
Acajutiba
Acarape
Acaraú
Acari
Acará
Acauã
Aceguá
Acopiara
Acorizal
Acrelândia
Acreúna
Acucena
Adamantina
Adelândia
Adolfo
Adrianópolis
Adustina
Afogados Da Ingazeira
Afonso Bezerra
Afonso Cláudio
Afonso Cunha
Afrânio
Afuá
Agrestina
Agricolândia
Agrolândia
Agronômica
Aguanil
Aguas Vermelhas
Aguaí
Agudo
Agudos
Agudos Do Sul
Aguiar
Aguiarnópolis
Aimorés
Aiquara
Aiuaba
Aiuruoca
Ajapi
Ajuricaba
Alagoa
Alagoa Grande
Alagoa Nova
Alagoinha
Alagoinha Do Piauí
Alagoinhas
Alambari
Albertina
Alcantil
Alcinópolis
Alcobaça
Alcântara
Alcântaras
Aldeias Altas
Alecrim
Alegre
Alegrete
Alegrete Do Piauí
Alegria
Alem Paraiba
Alenquer
Alexandria
Alexânia
Alfenas
Alfredo Chaves
Alfredo Marcondes
Alfredo Vasconcelos
Alfredo Wagner
Algodão De Jandaíra
Alhandra
Aliança
Aliança Do Tocantins
Almadina
Almas
Almeirim
Almenara
Almino Afon